# Prices tutorial

#### Sections
* [PricesYahoo](#PricesYahoo)
* [Symbols](#Symbols)
* [Calendars](#Calendars)
    * [`lead_symbol`](#lead_symbol)
    * [`calendars`](#calendars)
* [`delays`](#delays)

#### Notes
* The cell **outputs** shown in this tutorial are based on executing the cells at **2022-05-12 16:44 UTC**. Simply rerun the cells to bring any dynamic output up to date.

### Setup
Run following cell to import tutorial dependencies

In [2]:
from zoneinfo import ZoneInfo

## PricesYahoo

`market_prices` uses Prices classes to access price data. The default `PricesYahoo` class provides for getting price data 'out-the-box' from Yahoo APIs (see [disclaimers](../../README.md) section of README.md) via yahooquery.

In [3]:
from market_prices import PricesYahoo

## Symbols

The `PricesYahoo` class requires a single argument to define the symbols for which price data is to be requested. The symbols can be defined as a list of strings or a single comma-delimited string.

In [4]:
symbols = "MSFT, GOOG"
prices = PricesYahoo(symbols)
prices.symbols

['MSFT', 'GOOG']

Prices are available for any symbol for which price data is available on [yahoo finance](https://uk.finance.yahoo.com/), for example...

In [6]:
# Note: this cell might take a little while to execute (10 seconds maybe)
# given the number of different exchange calendars that will be created.
symbols = [
    "MSFT",  # us stock
    "AZN.L",  # uk stock
    "9988.HK",  # hong kong stock
    "PETR3.SA",  # brazilan stock
    "^FTSE",  # equity index
    "ES=F",  # futures
    "CL=F",  # oil
    "GC=F",  # gold
    "GBPEUR=X",  # currency pair
    "BTC-USD",  # crypto
]
prices = PricesYahoo(symbols, lead_symbol="9988.HK")

In [7]:
# last 30 mins of data at 10min intervals
prices.get("10min", minutes=30, tzout=ZoneInfo("UTC"))

symbol                                        9988.HK                        \
                                                close       high        low   
[2022-05-12 07:30:00, 2022-05-12 07:40:00)  80.000000  80.300003  79.900002   
[2022-05-12 07:40:00, 2022-05-12 07:50:00)  80.300003  80.349998  79.949997   
[2022-05-12 07:50:00, 2022-05-12 08:00:00)  79.949997  80.449997  79.800003   

symbol                                                             AZN.L  \
                                                 open     volume   close   
[2022-05-12 07:30:00, 2022-05-12 07:40:00)  80.199997  2499259.0  9801.0   
[2022-05-12 07:40:00, 2022-05-12 07:50:00)  80.000000  2462802.0  9862.0   
[2022-05-12 07:50:00, 2022-05-12 08:00:00)  80.349998  2616271.0  9824.0   

symbol                                                                       \
                                              high     low    open   volume   
[2022-05-12 07:30:00, 2022-05-12 07:40:00)  9821.0  9785.0  9799.0  55810.0   
[2022-05-12 07:40:00, 2022-05-12 07:50:00)  9862.0  9794.0  9799.0  39472.0   
[2022-05-12 07:50:00, 2022-05-12 08:00:00)  9866.0  9822.0  9863.0  39555.0   

symbol                                      ... PETR3.SA                       \
                                            ...    close high low open volume   
[2022-05-12 07:30:00, 2022-05-12 07:40:00)  ...      NaN  NaN NaN  NaN    NaN   
[2022-05-12 07:40:00, 2022-05-12 07:50:00)  ...      NaN  NaN NaN  NaN    NaN   
[2022-05-12 07:50:00, 2022-05-12 08:00:00)  ...      NaN  NaN NaN  NaN    NaN   

symbol                                            ^FTSE               \
                                                  close         high   
[2022-05-12 07:30:00, 2022-05-12 07:40:00)  7181.529785  7194.009766   
[2022-05-12 07:40:00, 2022-05-12 07:50:00)  7220.209961  7220.770020   
[2022-05-12 07:50:00, 2022-05-12 08:00:00)  7208.950195  7221.839844   

symbol                                                                       
                                                    low         open volume  
[2022-05-12 07:30:00, 2022-05-12 07:40:00)  7171.970215  7191.529785    0.0  
[2022-05-12 07:40:00, 2022-05-12 07:50:00)  7176.339844  7181.540039    0.0  
[2022-05-12 07:50:00, 2022-05-12 08:00:00)  7204.810059  7220.169922    0.0  

[3 rows x 50 columns]

Note that prices will show as missing for any instruments not trading over the requested period.

## Calendars

In order to evaluate the period over which data has been requested, `market_prices` requires that each symbol is associated with an exchange calendar of the library `exchange_calendars`. (See the [exchange_calendars](https://github.com/gerrymanoim/exchange_calendars) library for tutorials dedicated to exchange calendars.)

`PricesYahoo` will by default attempt to match each symbol with the exchange calendar that best reflects the symbol's trading times.

In [8]:
prices.calendars

{'MSFT': <exchange_calendars.exchange_calendar_xnys.XNYSExchangeCalendar at 0x1cff73d5400>,
 'AZN.L': <exchange_calendars.exchange_calendar_xlon.XLONExchangeCalendar at 0x1cf8a964af0>,
 '9988.HK': <exchange_calendars.exchange_calendar_xhkg.XHKGExchangeCalendar at 0x1cf8a964460>,
 'PETR3.SA': <exchange_calendars.exchange_calendar_bvmf.BVMFExchangeCalendar at 0x1cf8e30b400>,
 '^FTSE': <exchange_calendars.exchange_calendar_xlon.XLONExchangeCalendar at 0x1cf8a964af0>,
 'ES=F': <exchange_calendars.exchange_calendar_cmes.CMESExchangeCalendar at 0x1cf8e2f6310>,
 'CL=F': <exchange_calendars.us_futures_calendar.QuantopianUSFuturesCalendar at 0x1cf8a988f40>,
 'GC=F': <exchange_calendars.exchange_calendar_cmes.CMESExchangeCalendar at 0x1cf8e2f6310>,
 'GBPEUR=X': <exchange_calendars.weekday_calendar.WeekdayCalendar at 0x1cf8a988850>,
 'BTC-USD': <exchange_calendars.always_open.AlwaysOpenCalendar at 0x1cf8a990880>}

By default, requested periods are evaluated against the most common calendar (or if there is no single most common calendar then the calendar associated with the first symbol passed that's associated with one of the most common calendars).

### `lead_symbol`

Alternatively, the default calendar can be set by passing the `lead_symbol` option to the Prices class. This will set the default calendar to the calendar associated with the passed symbol. In the example above `lead_symbol` was passed as '9988.HK', which is the symbol for Alibaba's Hong Kong listing.

In [9]:
prices.calendar_default

The period over which prices were returned was therefore evaluated as the last 30 minutes over which the Hong Kong exchange was open.

Note: the lead symbol can be overriden for any particular call to `get`. The following example has the same arguments as earlier although passes `lead_symbol` as the symbol for Bitcoin. Bitcoin trades 24/7, hence the returned prices reflect the 30 minutes to the end of the current 'live indice' (times are UTC).

In [10]:
df = prices.get("10min", minutes=30, tzout=ZoneInfo("UTC"), lead_symbol='BTC-USD')
df

symbol                                     9988.HK                      AZN.L  \
                                             close high low open volume close   
[2022-05-12 16:20:00, 2022-05-12 16:30:00)     NaN  NaN NaN  NaN    NaN   NaN   
[2022-05-12 16:30:00, 2022-05-12 16:40:00)     NaN  NaN NaN  NaN    NaN   NaN   
[2022-05-12 16:40:00, 2022-05-12 16:50:00)     NaN  NaN NaN  NaN    NaN   NaN   

symbol                                                           ...  \
                                           high low open volume  ...   
[2022-05-12 16:20:00, 2022-05-12 16:30:00)  NaN NaN  NaN    NaN  ...   
[2022-05-12 16:30:00, 2022-05-12 16:40:00)  NaN NaN  NaN    NaN  ...   
[2022-05-12 16:40:00, 2022-05-12 16:50:00)  NaN NaN  NaN    NaN  ...   

symbol                                       PETR3.SA                        \
                                                close       high        low   
[2022-05-12 16:20:00, 2022-05-12 16:30:00)  36.419998  36.450001  36.250000   
[2022-05-12 16:30:00, 2022-05-12 16:40:00)  36.419998  36.419998  36.419998   
[2022-05-12 16:40:00, 2022-05-12 16:50:00)  36.419998  36.419998  36.419998   

symbol                                                          ^FTSE       \
                                                 open    volume close high   
[2022-05-12 16:20:00, 2022-05-12 16:30:00)  36.270000  220200.0   NaN  NaN   
[2022-05-12 16:30:00, 2022-05-12 16:40:00)  36.419998       0.0   NaN  NaN   
[2022-05-12 16:40:00, 2022-05-12 16:50:00)  36.419998       0.0   NaN  NaN   

symbol                                                      
                                           low open volume  
[2022-05-12 16:20:00, 2022-05-12 16:30:00) NaN  NaN    NaN  
[2022-05-12 16:30:00, 2022-05-12 16:40:00) NaN  NaN    NaN  
[2022-05-12 16:40:00, 2022-05-12 16:50:00) NaN  NaN    NaN  

[3 rows x 50 columns]

In [11]:
df["BTC-USD"]

,close,high,low,open,volume
"[2022-05-12 16:20:00, 2022-05-12 16:30:00)",29590.166016,29590.166016,29187.529297,29187.529297,673996800.0
"[2022-05-12 16:30:00, 2022-05-12 16:40:00)",29686.349609,29761.673828,29546.121094,29546.121094,777789440.0
"[2022-05-12 16:40:00, 2022-05-12 16:50:00)",29537.708984,29617.484375,29537.708984,29617.484375,0.0


(See the [periods tutorial](./periods.ipynp) tutorial for an explanation of how the requested period is evaluated.)

### `calendars`

If a calendar cannot be ascertained for a symbol then a `CalendarError` is raised. In this case it's necessary to manually assign calendars to those symbols for which a calendar cannot be ascertained. This is done by passing the `calendars` option to the Prices class. The `calendars` option can also be passed to override, for any symbol(s), the default calendar(s) that would otherwise be assigned.

`calendars` can take an `ExchangeCalendar`, a `list` or a `dict`, as described by the `PricesYahoo` class documentation (see final cell of this tutorial).

## `delays`

For each symbol `market_prices` also requires knowledge of any real-time delay in the price data (this is in order to evaluate periods to 'now').

By default, PricesYahoo attempts to evaluate the delay for each symbol via fields made available by the Yahoo API and some hardcoded mappings. A `ValueError` is raised if a delay cannot be ascertained for a symbol. In this case it's necessary to manually assign a delay via the `delays` kwarg. `delays` can also be passed to override the default delay that would otherwise be assigned to a specific symbol or symbols.

(NB an inaccurately evaluated delay can have the effect that the latest real-time prices 'stick' rather than update on further requests, or that data is unavailable at the most recent time for which it would be expected to be available.)

`delays` can take an `int`, a `list` or a `dict`, as described by the `PricesYahoo` class documentation...

In [ ]:
# or PricesYahoo?
help(PricesYahoo)